In [1]:
import pandas as pd
import numpy as np 

#required for building the interactive dashboard
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [2]:
#Reading The CSV file
df=pd.read_csv("/Users/vel/Desktop/projects/random_banking_transactions_with_categories.csv")

In [3]:
#Converting Date  into date data type to do further discrimination
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

In [5]:
# Get the latest month and year
latest_year = df[df.Year == df['Year'].max()]
latest_month = latest_year['Month'].max()

In [6]:
# Filter the dataframe to include only transactions from the latest month
last_month_expenses = df[(df['Month'] == latest_month)]
last_month_expenses

,Date,Transaction Type,Amount,Category,Month,Year
2,2023-07-28,withdrawal,-800.03,Entertainment,7,2023
17,2023-07-13,withdrawal,-277.35,Appliances,7,2023
20,2023-07-18,deposit,549.05,Entertainment,7,2023
25,2023-07-06,withdrawal,-445.60,Loans,7,2023
32,2023-07-20,deposit,990.74,Restaurant,7,2023
33,2023-07-10,deposit,497.20,Loans,7,2023
37,2023-07-20,deposit,162.93,Entertainment,7,2023
51,2023-07-16,deposit,104.26,Restaurant,7,2023
55,2023-07-11,deposit,700.55,Entertainment,7,2023
57,2023-07-28,withdrawal,-750.20,Other,7,2023


In [7]:
last_month_expenses = last_month_expenses.groupby('Category')['Amount'].sum().reset_index()

last_month_expenses['Amount']=last_month_expenses['Amount'].astype('str')
last_month_expenses['Amount']=last_month_expenses['Amount'].str.replace('-','')
last_month_expenses['Amount']=last_month_expenses['Amount'].astype('float')        #get absolute figures

last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Excluded|unassigned|Other") == False]    #exclude "excluded" category
last_month_expenses = last_month_expenses.sort_values(by='Amount', ascending=False)    #sort values
last_month_expenses['Amount'] = last_month_expenses['Amount'].round().astype(int)      #round values

print(last_month_expenses)

        Category  Amount
0     Appliances    1255
5     Restaurant    1095
2      Groceries     825
6         Travel     819
1  Entertainment     259
7      Utilities     168
3          Loans      63


In [8]:
last_month_expenses_tot = last_month_expenses['Amount'].sum()
print(last_month_expenses_tot)

4484


In [9]:
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")

income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")

damn=pn.Column(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget)


In [10]:
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='Amount', 
    height=250, 
    width=650, 
    title="Last Month Expenses",
    ylim=(0, 1000))

last_month_expenses_chart

:Bars   [Category]   (Amount)

In [11]:

df['Date'] = pd.to_datetime(df['Date'])            # convert the 'Date' column to a datetime object
df['Month-Year'] = df['Date'].dt.to_period('M')    # extract the month and year from the 'Date' column and create a new column 'Month-Year'
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('str')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].str.replace('-','')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("Other") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='Amount', ascending=False)
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'Amount': 'Amount '})

monthly_expenses_trend_by_cat

,Month-Year,Category,Amount
3,2023-01,Restaurant,1842
11,2023-02,Restaurant,1709
34,2023-05,Utilities,1697
12,2023-02,Travel,1616
4,2023-01,Travel,1454
32,2023-05,Restaurant,1415
5,2023-01,Utilities,1333
13,2023-02,Utilities,1283
41,2023-07,Appliances,1255
46,2023-07,Restaurant,1095


In [12]:
#Define Panel widget

select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All','Appliances', 'Groceries', 'Credit Cards', 'Loans', 'Entertainment', 'Utilities', 'Travel','Restaurant',
])
select_category1
# define plot function
def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Amount ')
    return plot
    
# define callback function
@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# create layout
monthly_expenses_trend_by_cat_chart = pn.Column(select_category1, update_plot)
monthly_expenses_trend_by_cat_chart[1].width = 600

monthly_expenses_trend_by_cat_chart

Column
    [0] Select(options=['All', 'Appliances', ...], value='All')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=600)

In [13]:
df = df[['Date', 'Category','Amount']]
df['Amount']=df['Amount'].astype('str')
df['Amount']=df['Amount'].str.replace('-','')
df['Amount']=df['Amount'].astype('float')        #get absolute figures

df = df[df["Category"].str.contains("Other") == False]    #exclude "excluded" category
df['Amount'] = df['Amount'].round().astype(int)      #round values
df['Date']=df['Date'].dt.date
df

,Date,Category,Amount
0,2023-06-01,Restaurant,915
1,2023-05-14,Loans,959
2,2023-07-28,Entertainment,800
3,2023-02-03,Entertainment,486
4,2023-05-19,Travel,321
...,...,...,...
94,2023-01-27,Credit Cards,706
95,2023-06-14,Credit Cards,500
97,2023-06-17,Travel,110
98,2023-03-08,Loans,357


In [14]:
# Define a function to filter the dataframe based on the selected category
def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]

# Create a DataFrame widget that updates based on the category filter
summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=400)

# Define a callback that updates the dataframe widget when the category filter is changed
def update_summary_table(event):
    summary_table.value = filter_df(event.new)

# Add the callback function to the category widget
select_category1.param.watch(update_summary_table, 'value')

summary_table

DataFrame(height=300, sizing_mode='fixed', value=          Date  ..., width=400)

In [17]:
# Create Final Dashboard

template = pn.template.FastListTemplate(
    title="Finance Summary Dashboard",
    theme='dark',
    sidebar_width=300,
    sidebar=[
        pn.pane.Markdown("## *Too many people spend money they earned..to buy things they don't want..to impress people that they don't like. --Will Rogers*"),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        damn
    ],
    main=[
        pn.Row(last_month_expenses_chart, sizingmode="stretch_both"),
        select_category1,
        pn.GridBox(
            monthly_expenses_trend_by_cat_chart[1],
            summary_table,
            ncols=2,
            width=500,  
            align='start',
            sizing_mode='stretch_width'
        )
    ]
)


template.show()

Launching server at http://localhost:58101
